# CS409 M: Chalk and Talk
---
## Harsh S Roniyar and Swarup Patil

---

## <u> Padding Oracle Attack on AES-CBC Mode </u>

### CBC Mode Encryption and Decryption

![image.png](./images/cbc_enc_dec.svg)

### Padding Scheme - PKCS #7

<!-- ![padding](pkcs7_pad.svg) -->
<img src="./images/pkcs7_pad.svg" alt="drawing" width="800"/>

### The Oracle

![oracle](./images/oracle.svg)

### Importing the Oracle

In [1]:
from oracle import encrypt, is_padding_ok, BLOCK_SIZE, _decrypt

### The Attack

In [2]:
def attack(ciphertext):
    guessed_clear = b""

    split_string = lambda x, n: [x[i : i + n] for i in range(0, len(x), n)]
    blocks = split_string(ciphertext, BLOCK_SIZE)

    for block_n in range(len(blocks) - 1, 0, -1):  # build pair of blocks starting from end of message
        spliced_ciphertext = blocks[block_n - 1] + blocks[block_n]

        decoded_bytes = b"?" * BLOCK_SIZE  # output of block cipher decoding values

        ##GET VALUE OF SECRET BYTE byte
        for byte in range(BLOCK_SIZE - 1, -1, -1):
            new_pad_len = BLOCK_SIZE - byte

            # Build hacked ciphertext tail with values to obtain desired padding
            hacked_ciphertext_tail = b""
            for padder_index in range(1, new_pad_len):
                hacked_ciphertext_tail += bytearray.fromhex(
                    "{:02x}".format(new_pad_len ^ decoded_bytes[byte + padder_index])
                )

            for i in range(0, 256):
                attack_str = bytearray.fromhex(
                    "{:02x}".format((i ^ spliced_ciphertext[byte]))
                )
                hacked_ciphertext = (
                    spliced_ciphertext[:byte]
                    + attack_str
                    + hacked_ciphertext_tail
                    + spliced_ciphertext[byte + 1 + new_pad_len - 1 :]
                )
                # print(test_correctness)
                
                # print("Hacked CipherText:", hacked_ciphertext[:BLOCK_SIZE].hex())
                if is_padding_ok(hacked_ciphertext):

                    test_correctness = (
                        hacked_ciphertext[: byte - 1]
                        + bytearray.fromhex(
                            "{:02x}".format((1 ^ hacked_ciphertext[byte - 1]))
                        )
                        + hacked_ciphertext[byte:]
                    )
                    
                    # print("Test Correctness", i, "+ byte:" , byte, "+", test_correctness[:BLOCK_SIZE].hex())
                    if not is_padding_ok(test_correctness):
                        continue

                    decoded_bytes = (
                        decoded_bytes[:byte]
                        + bytearray.fromhex(
                            "{:02x}".format(hacked_ciphertext[byte] ^ new_pad_len)
                        )
                        + decoded_bytes[byte + 1 :]
                    )
                    guessed_clear = (
                        bytearray.fromhex("{:02x}".format(i ^ new_pad_len))
                        + guessed_clear
                    )

                    # print(guessed_clear)
                    break

    return guessed_clear
    # return guessed_clear[:-guessed_clear[-1]] #remove padding!

### Performing the Attack

In [3]:
def test_the_attack():
    messages = (
        b"Attack at dawn",
        b"",
        b"Giovanni",
        b"In symmetric cryptography, the padding oracle attack can be applied to the CBC mode of operation,"
        + b'where the "oracle" (usually a server) leaks data about whether the padding of an encrypted '
        + b"message is correct or not. Such data can allow attackers to decrypt (and sometimes encrypt) "
        + b"messages through the oracle using the oracle's key, without knowing the encryption key",
        b"I am a happy fellow dancing under the sun",
    )
    for msg in messages:
        print("-" * 128)
        print("Testing :", msg, "OF LENGTH", len(msg))
        print("The encrypted message is :", encrypt(msg))
        cracked_ct = attack(encrypt(msg))
        print("The decrypted message (with padding) is :", cracked_ct)
        print("-" * 128)
        cracked_nopad = cracked_ct[: -cracked_ct[-1]]
        if cracked_nopad == msg:
            print("Assertion passed!")
        else:
            print("Assertion failed!")
        print("-" * 128)

In [4]:
if __name__ == "__main__":
    test_the_attack()

--------------------------------------------------------------------------------------------------------------------------------
Testing : b'Attack at dawn' OF LENGTH 14
The encrypted message is : b"Xi\x95RA\xb3\x97v,\xb9\x91\xe3;\xfa'\xd5n\xfcLT@\xb2\xa8\xf2\xda}\xceqM\xfff\xda"
The decrypted message (with padding) is : bytearray(b'Attack at dawn\x02\x02')
--------------------------------------------------------------------------------------------------------------------------------
Assertion passed!
--------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------
Testing : b'' OF LENGTH 0
The encrypted message is : b'\xcd\xca\xbdIB\xe9\xde\xfd\xda\xb7#A-X\xe9!\x9c\xa6\x98f\xab\xb1\x07\xbe\n\x9928\x8c\xdc*\xe7'
The decrypted message (with padding) is : bytearray(b'\x10\x10\x10\x10\x10\x10\x10\x1